# Linear Optimization (CS5040) Assignment 4

## Authors

| Name | Roll Number |
|-|-|
| Gautam Singh | CS21BTECH11018 |
| Varun Gupta | CS21BTECH11060 |
| Anshul Sangrame | CS21BTECH11004 |

## Setup

In [6]:
# Install libraries
%pip install numpy

# Import libraries
import numpy as np

Note: you may need to restart the kernel to use updated packages.


In [7]:
# Parameters to run the program go here
INPUT_FILE = '../data/input_01.csv'    # Input file path
DELIMITER = ','                     # Delimiter in input file

## Input Handling

In [8]:
def handle_input(
    fname: str, 
    delimiter: str=',',
) -> (np.ndarray, np.ndarray, np.ndarray, np.ndarray):
    """
    Handle input from CSV file.
    """
    # Take input from CSV file into numpy array
    input_arr = np.genfromtxt(INPUT_FILE, delimiter=DELIMITER, skip_header=0)

    # Values of A, b, c, z
    A = input_arr[2:, :-1]
    b = input_arr[2:, -1]
    c = input_arr[1, :-1]
    z = input_arr[0, :-1]

    # Check for bad inputs, and exit if found
    if np.any(np.isnan(A)):
        raise IOError('Matrix A contains bad input:', A)
    if np.any(np.isnan(b)):
        raise IOError('Matrix b contains bad input:', b)
    if np.any(np.isnan(c)):
        raise IOError('Matrix c contains bad input:', c)
    if np.any(np.isnan(z)):
        raise IOError('Matrix z contains bad input:', z)
    # Values of m and n
    m, n = A.shape
    # Check if A is full rank
    if np.linalg.matrix_rank(A) != n:
        raise np.linalg.LinAlgError('Matrix A is not full rank:', A)
    return A, b, c, z

## Finding the Optimal Vertex

In [9]:
def vertex_directions(
    A: np.ndarray,
    b: np.ndarray,
    c: np.ndarray,
    v: np.ndarray,
) -> np.ndarray:
    """
    Function to find directions of the other vertices of the polytope from given
    vertex.
    """
    tight_rows = np.where(np.isclose(A@v, b))
    A1 = A[tight_rows]
    return -np.linalg.inv(A1.T)

def simplex_neighbour(
    A: np.ndarray,
    b: np.ndarray,
    c: np.ndarray,
    u: np.ndarray,
) -> np.ndarray | None:
    """
    Function to find a neighbouring vertex with greater cost, or report that
    there is no such neighbour.
    """
    # Find directions to other vertices
    z = vertex_directions(A, b, c, u)

    # Find costs for each direction
    costs = z@c

    # Find directions which give positive cost
    costs_positive = np.where(costs > 0)[0]
    
    # If there are no such directions, declare optimality
    if len(costs_positive) == 0:
        return None
    else:
        # Get any direction with positive cost
        v = z[costs_positive[0]]

        # Check for unboundedness. If A@v keeps decreasing in that direction,
        # then the LP is unbounded.
        print(v, A@v)
        if len(np.where(A@v > 0)) == 0:
            raise np.linalg.LinAlgError('LP is unbounded.')

        # Find untight rows
        untight_rows = np.where(~np.isclose(A@u, b))
        A2 = A[untight_rows]
        b2 = b[untight_rows]

        # Find feasible neighbour and required coefficients
        # Coefficients are (b2 - A2@u)/(A2@v)
        alpha = (b2-A2@u)/(A2@v)
        t = np.min(alpha[alpha >= 0])
        return u + t*v

def simplex(
    A: np.ndarray,
    b: np.ndarray,
    c: np.ndarray,
    u: np.ndarray,
    n_iter: int=1000,
) -> np.ndarray:
    """ 
    Function to implement the simplex algorithm.
    """
    while n_iter:
        # Display vertex and cost
        print(u, c.T@u)
        # Find neighbour of a greater cost
        u1 = simplex_neighbour(A, b, c, u)
        if u1 is None:
            return u
        else:
            u = u1
        n_iter -= 1

In [10]:
if __name__ == "__main__":
    A, b, c, z = handle_input(INPUT_FILE, DELIMITER)
    # Check if z is feasible
    if not np.all(A@z <= b):
        raise RuntimeError('Given starting point', z, 'is not feasible.')
    simplex(A, b, c, z)

[10.  0.] 20.0
[30. 20.] 80.0


ValueError: zero-size array to reduction operation minimum which has no identity